In [1]:
import numpy as np
import os
import sys
import tensorflow as tf

sys.path.append("..")
from utils import label_map_util
from utils import visualization_utils as vis_util

########## For TF2.1 ############
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#################################

import cv2
from time import time

W0318 07:51:34.212156  7752 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\tf_detection\lib\site-packages\tensorflow_core\python\compat\v2_compat.py:88: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [14]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(r"D:\Courses\pyimagesearch\real-time object detection\real-time-object-detection\external data\car_chase_02.mp4")

Choose a Model

In [15]:
# MODEL_NAME = 'ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03'
# MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
# MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
# MODEL_NAME = 'ssd_mobilenet_v2_oid_v4_2018_12_12'
# MODEL_NAME = 'ssdlite_mobilenet_v2_coco_2018_05_09'
MODEL_NAME = 'trt_ssdlite'

In [16]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')


detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [17]:
# ## Loading label map
# Label maps map indices to category names, so that when our convolution network predicts `5`, 
# we know that this corresponds to `airplane`.  Here we use internal utility functions, 
# but anything that returns a dictionary mapping integers to appropriate string labels would be fine
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [18]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [19]:
times = []

start_time = time()
frame_counter = 0

# Detection
with tf.device('/gpu:0'):
  with detection_graph.as_default():
      with tf.Session(graph=detection_graph) as sess:
          ret = True
          while ret:
              # Read frame from camera
              ret, image_np = cap.read()
              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
              image_np_expanded = np.expand_dims(image_np, axis=0)
              # Extract image tensor
              image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
              # Extract detection boxes
              boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
              # Extract detection scores
              scores = detection_graph.get_tensor_by_name('detection_scores:0')
              # Extract detection classes
              classes = detection_graph.get_tensor_by_name('detection_classes:0')
              # Extract number of detectionsd
              num_detections = detection_graph.get_tensor_by_name(
                  'num_detections:0')
            
              # Actual detection.
                
              t1 = time()
            
              (boxes, scores, classes, num_detections) = sess.run(
                  [boxes, scores, classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
                
              t2 = time()
              times.append(t2-t1)
              times = times[-20:]
            
              # Visualization of the results of a detection.
              vis_util.visualize_boxes_and_labels_on_image_array(
                  image_np,
                  np.squeeze(boxes),
                  np.squeeze(classes).astype(np.int32),
                  np.squeeze(scores),
                  category_index,
                  use_normalized_coordinates=True,
                  line_thickness=8)

              # Display output
              image_np = cv2.putText(image_np, "Time: {:.2f}ms".format(sum(times)/len(times)*1000), (0, 30),
                                cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
              cv2.imshow('object detection', cv2.resize(image_np, (800, 600)))

              frame_counter = frame_counter + 1

              if cv2.waitKey(1) & 0xFF == ord('q'):
                  break


cap.release()
cv2.destroyAllWindows()
seconds = time() - start_time
if seconds > 0:
    print('fps:', frame_counter/seconds)
else:
    print(f'seconds = {seconds}')


fps: 39.24246093954543
